<a href="https://colab.research.google.com/github/temur-shd/GloveTrainingOwnCorpus/blob/main/GloveTrainingImdb250.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab'e klonluyoruz
!git clone https://github.com/stanfordnlp/glove

Cloning into 'glove'...
remote: Enumerating objects: 631, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 631 (delta 21), reused 30 (delta 21), pack-reused 592
Receiving objects: 100% (631/631), 235.75 KiB | 10.25 MiB/s, done.
Resolving deltas: 100% (359/359), done.


In [ ]:
Repo building kısmından önce klasördeki demo.sh dosyasını kendi korpusumuza göre düzenliyoruz.
-----------------------------------------------------------------------
kalın metin

#!/bin/bash
set -e

# Makes programs, downloads sample data, trains a GloVe model, and then evaluates it.
# One optional argument can specify the language used for eval script: matlab, octave or [default] python

#data.txt'nin lokasyonunu veriyoruz
CORPUS=/content/data.txt
#modelin oluşturacağı dosyaları saklayacak bir folder yaratıyoruz ve buraya veriyoruz ben turkish_glove olsun dedim
VOCAB_FILE=/content/turkish_glove/vocab.txt
COOCCURRENCE_FILE=/content/turkish_glove/cooccurrence.bin
COOCCURRENCE_SHUF_FILE=/content/turkish_glove/cooccurrence.shuf.bin
SAVE_FILE=/content/turkish_glove/vectors

#glove klasöründeki build klasörünün yerini veriyoruz
BUILDDIR=/content/glove/build


#Hyperparameterleri değiştirebilirsin 
VERBOSE=2
MEMORY=4.0
VOCAB_MIN_COUNT=1
VECTOR_SIZE=50
MAX_ITER=15
WINDOW_SIZE=15
BINARY=2
NUM_THREADS=8
X_MAX=10
if hash python 2>/dev/null; then
    PYTHON=python
else
    PYTHON=python3
fi


echo
echo "$ $BUILDDIR/vocab_count -min-count $VOCAB_MIN_COUNT -verbose $VERBOSE < $CORPUS > $VOCAB_FILE"
$BUILDDIR/vocab_count -min-count $VOCAB_MIN_COUNT -verbose $VERBOSE < $CORPUS > $VOCAB_FILE
echo "$ $BUILDDIR/cooccur -memory $MEMORY -vocab-file $VOCAB_FILE -verbose $VERBOSE -window-size $WINDOW_SIZE < $CORPUS > $COOCCURRENCE_FILE"
$BUILDDIR/cooccur -memory $MEMORY -vocab-file $VOCAB_FILE -verbose $VERBOSE -window-size $WINDOW_SIZE < $CORPUS > $COOCCURRENCE_FILE
echo "$ $BUILDDIR/shuffle -memory $MEMORY -verbose $VERBOSE < $COOCCURRENCE_FILE > $COOCCURRENCE_SHUF_FILE"
$BUILDDIR/shuffle -memory $MEMORY -verbose $VERBOSE < $COOCCURRENCE_FILE > $COOCCURRENCE_SHUF_FILE
echo "$ $BUILDDIR/glove -save-file $SAVE_FILE -threads $NUM_THREADS -input-file $COOCCURRENCE_SHUF_FILE -x-max $X_MAX -iter $MAX_ITER -vector-size $VECTOR_SIZE -binary $BINARY -vocab-file $VOCAB_FILE -verbose $VERBOSE"
$BUILDDIR/glove -save-file $SAVE_FILE -threads $NUM_THREADS -input-file $COOCCURRENCE_SHUF_FILE -x-max $X_MAX -iter $MAX_ITER -vector-size $VECTOR_SIZE -binary $BINARY -vocab-file $VOCAB_FILE -verbose $VERBOSE
if [ "$CORPUS" = 'text8' ]; then
   if [ "$1" = 'matlab' ]; then
       matlab -nodisplay -nodesktop -nojvm -nosplash < ./eval/matlab/read_and_evaluate.m 1>&2 
   elif [ "$1" = 'octave' ]; then
       octave < ./eval/octave/read_and_evaluate_octave.m 1>&2
   else
       echo "$ $PYTHON eval/python/evaluate.py"
       $PYTHON eval/python/evaluate.py
   fi
fi

In [ ]:
#Reponun buildini yapıyoruz
!cd /content/glove && make

mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/cooccur.c: In function ‘merge_files’:
src/cooccur.c:180:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
  180 |         fread(&new, sizeof(CREC), 1, fid[i]);
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:190:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
  190 |     fread(&new, sizeof(CREC), 1, fid[i]);
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:203:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
  203 |         fread(&new, sizeof(CREC), 1, fid[i]);
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gcc -c src/shuffle.c -o b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#t_imdb250.txt'nin içinde olduğu klasörü gdriverdan çekiyoruz
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r '/content/gdrive/MyDrive/turkish_glove' '/content/'
drive.flush_and_unmount()

Mounted at /content/gdrive


In [ ]:
#t_imdb250'dakı satırlarla ayrılmış datayı data.txt'de sadece aralarında boşluk olacak şekilde ayırıyoruz
datas=open("/content/turkish_glove/t_imdb250.txt").readlines()
f=open("data.txt","w")
for i in datas:
  f.write(i.strip()+" ")

In [ ]:
!/content/glove/demo.sh 


$ /content/glove/build/vocab_count -min-count 1 -verbose 2 < /content/data.txt > /content/turkish_glove/vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.Processed 1121384 tokens.
Counted 119192 unique words.
Using vocabulary of size 119192.

$ /content/glove/build/cooccur -memory 4.0 -vocab-file /content/turkish_glove/vocab.txt -verbose 2 -window-size 15 < /content/data.txt > /content/turkish_glove/cooccurrence.bin
COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "/content/turkish_glove/vocab.txt"...loaded 119192 words.
Building lookup table...table contains 109079219 elements.
Processing token: 010000020000030000040000050000060000070000080000090000010000001100000Processed 1121384 tokens.
Writing cooccurrences to disk.........2 files in total.
Merging 

In [ ]:
#turkish_glove klasörünü daha sonra kolayca çekmek için gdrive'a atıyorum
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/turkish_glove /content/gdrive/MyDrive
drive.flush_and_unmount()

Mounted at /content/gdrive
